In [1]:
import os

In [2]:
%pwd

'/Users/bhikipallai/Desktop/Projects/95Mobiles/research/bike'

In [3]:
os.chdir("../../")

In [4]:
%pwd

'/Users/bhikipallai/Desktop/Projects/95Mobiles'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class Bike_feature_engg:
    root_dir: Path
    bike_raw_data: Path
    feature_eng_data: Path

In [6]:
from src.bike.utils.common import read_yaml,create_directory
from src.bike.constants import *

[2024-10-02 19:03:16,259: INFO]: Logging file start


In [7]:
class ConfigurationManger:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,param_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(param_filepath)

        create_directory([self.config.data_root])

    def get_feature_eng_config(self) -> Bike_feature_engg:
        config = self.config.feature_engeenering

        create_directory([config.root_dir])

        feature_config = Bike_feature_engg(
            root_dir= config.root_dir,
            bike_raw_data=config.bike_raw_data,
            feature_eng_data = config.feature_eng_data
        )

        return feature_config

In [8]:
import pandas as pd
import numpy as np

In [9]:
from sklearn.preprocessing import LabelEncoder,StandardScaler

In [10]:
class feature_eng:
    cat_cols = []
    num_cols = []
    def __init__(self,config:Bike_feature_engg):
        self.config = config

    def load_data(self):
        self.df = pd.read_csv(self.config.bike_raw_data)
        return self.df
    
    def datatypechange(self):
        self.df['power'] = self.df['power'].astype(int)
        print(self.df['power'].dtypes)
    
    def make_list_as_per_object(self):
        for cols in self.df.columns:
            if self.df[cols].dtypes == 'float64' or self.df[cols].dtypes == 'int64':
                feature_eng.num_cols.append(cols)
            elif self.df[cols].dtypes == 'object':
                feature_eng.cat_cols.append(cols)
        print(feature_eng.cat_cols,feature_eng.num_cols)

    def short_the_city_column(self):
        for city,count in self.df['city'].value_counts().items():
            if count <= 900:
                self.df['city'] = self.df['city'].replace(city,"others")
        print(self.df['city'].nunique())
    
    def encode_cat_cols(self):
        le = LabelEncoder()
        for cols in feature_eng.cat_cols:
            self.df[cols] = le.fit_transform(self.df[cols])
            print(self.df.head(2))
    
    def encode_num_cols(self):
        stc = StandardScaler()
        feature_eng.num_cols.remove('price')
        feature_eng.num_cols.remove('age')
        for cols in feature_eng.num_cols:
            self.df[cols] = stc.fit_transform(self.df[[cols]])
        print(self.df.head(2))

    def save_the_transformed_data(self):
        self.df.to_csv(self.config.feature_eng_data)

In [11]:
try:
    config = ConfigurationManger()
    fe_config = config.get_feature_eng_config()
    fe = feature_eng(config=fe_config)
    fe.load_data()
    fe.datatypechange()
    fe.make_list_as_per_object()
    fe.short_the_city_column()
    fe.encode_cat_cols()
    fe.encode_num_cols()
    fe.save_the_transformed_data()
except Exception as e:
    raise e

[2024-10-02 19:03:17,362: INFO]: yaml: config/bike_config.yaml loaded successfully
[2024-10-02 19:03:17,364: INFO]: yaml: params/bike_params.yaml loaded successfully
[2024-10-02 19:03:17,364: INFO]: created directory at: data/bike
[2024-10-02 19:03:17,365: INFO]: created directory at: data/bike/feature_engeenering
int64
['bike_name', 'city', 'owner', 'brand'] ['price', 'kms_driven', 'age', 'power']
11
   bike_name     price       city  kms_driven        owner  age  power  \
0        401   35000.0  Ahmedabad     17654.0  First Owner  3.0    110   
1        295  119900.0      Delhi     11000.0  First Owner  4.0    350   

           brand  
0            TVS  
1  Royal Enfield  
   bike_name     price  city  kms_driven        owner  age  power  \
0        401   35000.0     0     17654.0  First Owner  3.0    110   
1        295  119900.0     3     11000.0  First Owner  4.0    350   

           brand  
0            TVS  
1  Royal Enfield  
   bike_name     price  city  kms_driven  owner  a